In [3]:
!pip install torch==1.6.0
!pip install opencv-python
!pip install torchvision==0.2.2
!pip install albumentations
!pip install tensorflow
!pip install pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 748.8 MB 16 kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 49.5 MB 29.4 MB/s 
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 5.6 MB/s 
     |████████████████████████████████| 52 kB 3.7 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user i

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import typing as tp
import yaml
import random
import os
import sys
import soundfile as sf
import librosa
import cv2
import matplotlib.pyplot as plt
import time
import glob
from tqdm import tqdm

import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# import resnest.torch as resnest_torch

from torchvision import models

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
# from resnet import ResNet, Bottleneck

from albumentations.core.transforms_interface import DualTransform, BasicTransform
import albumentations as albu

from sklearn.model_selection import StratifiedKFold

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

## util

In [2]:
config_set = {
    'dataset': {
          'name': 'SpectrogramDataset',
          'params': {
            'img_size': 224, 
            'melspectrogram_parameters': {
                'n_fft': 1024,
                'hop_length': 320,
                'win_length': 1024,
                'n_mels': 64, 
                'fmin': 50, 
                'fmax': 24000, 
            }
      }
    },
    'loader': {
      'train': {
        'batch_size': 6,
        'shuffle': True,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      },
      'valid': {
        'batch_size': 2,
        'shuffle': False,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      }
    }
}
SEED=1213
PERIOD = 10
SPECIES_NUM = 24
EPOCH = 50
HOP_LEN = 512
SR = 48000
BATCH = 2

In [3]:
config = config_set

In [4]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
set_seed(SEED)

In [5]:
INPUT_ROOT = Path("/home/knikaido/work/Rainforest-Connection/data")
RAW_DATA = INPUT_ROOT / "rfcx-species-audio-detection"
TRAIN_AUDIO_DIR = RAW_DATA / "train"
# TRAIN_RESAMPLED_AUDIO_DIRS = [
#   INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
# ]
TEST_AUDIO_DIR = RAW_DATA / "test"
OUTPUT_DIR = './output/'

In [6]:
train_gby = pd.read_pickle(RAW_DATA / "train_gby_mel.pkl")
train_gby.head()

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name
0,003bec244,[14],[1],[44.544],[2531.25],[45.1307],[5531.25],/home/knikaido/work/Rainforest-Connection/Git/...
1,006ab765f,[23],[1],[39.9615],[7235.16],[46.0452],[11283.4],/home/knikaido/work/Rainforest-Connection/Git/...
2,007f87ba2,[12],[1],[39.135999999999996],[562.5],[42.272],[3281.25],/home/knikaido/work/Rainforest-Connection/Git/...
3,0099c367b,[17],[4],[51.4206],[1464.26],[55.1996],[4565.04],/home/knikaido/work/Rainforest-Connection/Git/...
4,009b760e6,[10],[1],[50.0854],[947.461],[52.5293],[10852.7],/home/knikaido/work/Rainforest-Connection/Git/...


In [7]:
def mono_to_color(
    X: np.ndarray, mean=None, std=None,
    norm_max=None, norm_min=None, eps=1e-6
):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [8]:
def get_criterion():
    pos_weights = torch.ones(SPECIES_NUM)
    pos_weights = pos_weights * SPECIES_NUM
    loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
    return loss_function

In [9]:
def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.

    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

In [10]:
class LitModule(pl.LightningModule):
    
    def __init__(self, train_len, mel_params, num_classes=SPECIES_NUM):
        super().__init__()
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.interpolate_ratio = 30  # Downsampled ratio
        # load pretrained models, using ResNeSt-50 as an example
        base_model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)
        layers = list(base_model.children())[:-1]
        self.encoder = nn.Sequential(*layers)

        in_features = 1024

        self.fc1 = nn.Linear(in_features, in_features, bias=True)
        self.att_block = AttBlock(in_features, num_classes, activation="sigmoid")
        
        
        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=mel_params['n_fft'], hop_length=mel_params['hop_length'], 
            win_length=mel_params['win_length'], window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=SR, n_fft=mel_params['n_fft'], 
            n_mels=mel_params['n_mels'], fmin=mel_params['fmin'], fmax=mel_params['fmax'], ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(mel_params['n_mels'])
        self.init_weight()

        self.init_weight()
#         self.criterion = bce_criterion
        self.train_len = train_len
        
    def init_weight(self):
        init_layer(self.fc1)
        init_bn(self.bn0)

    def forward(self, x, istrain=True, mixup_lambda=None):
        
#         if self.training and mixup_lambda is not None:
#             x = do_mixup(x, mixup_lambda)
        
        x = self.spectrogram_extractor(x)
        # batch_size x 1 x time_steps x freq_bins
        x = self.logmel_extractor(x)
        # batch_size x 1 x time_steps x mel_bins

        frames_num = x.shape[2]

        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        #print(x.shape)
        
#         if istrain:
#             x = self.spec_augmenter(x)
#         if istrain and mixup_lambda is not None  :
#             x = do_mixup(x, mixup_lambda)

        # Output shape (batch size, channels, time, frequency)
        x = x.expand(x.shape[0], 3, x.shape[2], x.shape[3])
        
        x = self.encoder(x)

        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)

        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "logit": logit,
            "clipwise_output": clipwise_output
        }

        return output_dict
    
    def configure_optimizers(self):
#         optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=0.01, momentum=0.9)
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=True)
#         scheduler = {'scheduler': optim.lr_scheduler.OneCycleLR(
#                                                 optimizer,
#                                                 max_lr=self.learning_rate,
#                                                 steps_per_epoch=int(len(self.train_dataloader())),
#                                                 epochs=self.hparams.epochs,
#                                                 anneal_strategy="linear",
#                                                 final_div_factor = 30,
#                                             ),
#                                 'name': 'learning_rate',
#                                 'interval':'step',
#                                 'frequency': 1}
        scheduler = {'scheduler': torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001*0.01),
                                'name': 'learning_rate',
                                'interval':'epoch',
                                'frequency': 1}
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#         lr_scheduler = {"scheduler": scheduler }
        return [optimizer], [scheduler]
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x, y_a, y_b, lam = mixup_data(x, y, alpha=0.5)
#         mixup_lambda = self.mixup_augmenter.get_lambda(6).to("cuda:0")
#         y = do_mixup(y, mixup_lambda)
        y_pred = self.forward(x, istrain=True)
        loss = mixup_criterion(self.criterion, y_pred['clipwise_output'], y_a, y_b, lam)
#         print(y_pred['clipwise_output'].shape, y.shape)
#         loss = self.criterion(y_pred['clipwise_output'], y)
        self.log('train_loss', loss,  on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_pred = self.forward(x, istrain=False)
#         print(y_pred)
        loss_max = self.criterion(torch.max(y_pred['framewise_output'], 1)[0], y)
        loss_clip = self.criterion(y_pred['clipwise_output'], y)
        loss = loss_clip + 0.5*loss_max
        y_pred_act = y_pred['clipwise_output']
        lwlap_step, weight_step = lwlap_wrapper(y, y_pred_act)
        lwlap_step = (lwlap_step * weight_step).sum()
#         print('valid_epoch_loss = ', loss)
#         print('valid_epoch_loss = ', lwlap_step)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('lwlap_score', lwlap_step, on_epoch=True, prog_bar=True, logger=True)
        return loss, lwlap_step

    def validation_epoch_end(self, validation_step_outputs):
        validation_step_outputs = np.array(validation_step_outputs)
        validation_step_losses = validation_step_outputs[:, 0]
        mean_loss = torch.stack([x for x in validation_step_losses]).mean()
        
        validation_step_scores = validation_step_outputs[:, 1]
        mean_score = torch.stack([x for x in validation_step_scores]).mean()

        print('valid_epoch_loss = ', mean_loss)
        print('valid_epoch_lwlap = ', mean_score)
        self.log('valid_epoch_loss', mean_loss, prog_bar=True, logger=True)
        self.log('valid_epoch_lwlap', mean_score, prog_bar=True, logger=True)
#         tqdm.write('Dice: \t%.3f' % mean_loss)
        return mean_loss, mean_score

In [11]:
model = LitModule(0, config['dataset']['params']['melspectrogram_parameters'])

Using cache found in /home/user/.cache/torch/hub/pytorch_vision_v0.6.0


In [12]:
model.load_state_dict(torch.load(OUTPUT_DIR + 'model'))

<All keys matched successfully>

In [13]:
def signal_to_mel(y, sr, mel_params):
    
    len_y = len(y)
    effective_length = int(SR * PERIOD)
    
    start = 0
    end = start + effective_length
    
    images = []
    while(start < len_y):
        if(end > len_y):
            break
        y_ele = y[start:end]
        
        images.append(y_ele.astype(np.float32))
        
        start = end
        end += effective_length
                
    return np.array(images)

In [14]:
test_wav_pathes = sorted(glob.glob(str(TEST_AUDIO_DIR / '*.flac')))
len(test_wav_pathes)

1992

In [15]:
device = torch.device("cuda")
model.eval().to(device)
preds = []

for path in tqdm(test_wav_pathes):
    y, sr = sf.read(path)
    mel_img = signal_to_mel(y, sr, config["dataset"]["params"])
    
    b_start = 0
    b_end = BATCH
    b_preds = []
    while(b_start < mel_img.shape[0]):
        if(b_end >= mel_img.shape[0]):
            mel_batch = mel_img[b_start:]
        else:
            mel_batch = mel_img[b_start:b_end]
        mel_batch = torch.from_numpy(mel_batch).clone().to(device)
        pred = model(mel_batch, istrain=False)
        pred = pred['clipwise_output'].to('cpu').detach().numpy().copy()
        b_preds.extend(pred)
        b_start += BATCH
        b_end += BATCH
    pred = np.mean(b_preds, axis=0)
    preds.append(pred)    
            
preds = np.array(preds)
#     break

100%|██████████| 1992/1992 [02:30<00:00, 13.22it/s]


In [16]:
sub = pd.read_csv(str(RAW_DATA / 'sample_submission.csv'))

In [17]:
sub.loc[:, 's0':'s23'] = preds

In [18]:
sub.to_csv(OUTPUT_DIR + '92_sub.csv', index=False)

In [19]:
sub

,recording_id,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
0,000316da7,1.190787e-03,1.752538e-08,1.736093e-07,0.037427,1.729519e-06,1.112790e-05,6.435797e-02,6.969092e-07,1.175076e-06,9.722112e-08,2.420424e-08,3.816639e-07,3.351258e-01,1.588973e-06,2.860986e-06,2.914543e-07,6.104043e-06,2.801543e-07,2.581988e-02,6.948874e-08,9.387915e-08,4.949152e-06,1.963555e-07,5.571133e-06
1,003bc2cb2,4.302320e-12,2.689255e-09,5.879835e-15,0.025863,5.611078e-14,5.054421e-14,1.949314e-15,1.207746e-11,4.855316e-14,1.179405e-07,6.406305e-11,2.813352e-13,5.718354e-11,7.136406e-12,4.755510e-14,4.793173e-11,9.737549e-01,1.304647e-08,6.802316e-16,2.328706e-13,3.004652e-12,1.058822e-08,4.263843e-14,1.383149e-12
2,0061c037e,1.760499e-05,6.017978e-05,1.228253e-05,0.001041,8.551092e-03,1.134303e-04,2.127888e-05,2.543979e-05,1.979750e-07,2.575553e-05,4.917106e-02,4.941571e-05,1.556362e-05,1.087925e-05,3.246694e-07,2.733840e-05,5.304927e-06,2.211083e-03,3.064391e-06,3.278762e-03,2.938364e-04,1.390236e-06,4.664826e-04,1.378366e-04
3,010eb14d3,9.583182e-01,1.451894e-08,8.119231e-09,0.000156,4.109373e-09,2.378589e-08,4.005762e-08,3.802381e-08,5.222932e-02,1.091532e-07,2.218200e-08,9.702507e-10,5.850231e-06,4.421263e-08,3.971600e-07,5.930083e-08,1.909815e-10,3.461194e-08,2.297185e-09,4.725277e-08,5.793869e-10,1.206597e-08,1.374843e-07,1.105585e-08
4,011318064,4.277624e-06,5.607159e-06,2.768406e-05,0.027513,1.866484e-06,1.273762e-04,6.898747e-05,1.663071e-05,8.900940e-07,2.473133e-06,6.424068e-06,1.476852e-05,8.648235e-05,1.220102e-05,6.339967e-01,1.520094e-01,2.844360e-05,8.777956e-05,7.447410e-02,8.744725e-06,1.512257e-06,1.885610e-04,8.539203e-06,1.781476e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,ff68f3ac3,1.022669e-03,2.114259e-08,1.568499e-04,0.079419,2.225942e-07,4.007234e-01,1.649217e-06,8.441630e-03,8.807204e-06,1.307804e-05,1.367811e-06,4.674571e-07,1.506697e-02,1.337271e-06,1.410481e-07,2.588280e-01,5.348056e-07,2.183345e-06,9.886725e-07,2.584863e-03,1.152750e-04,6.979222e-06,1.638479e-05,2.774945e-01
1988,ff973e852,2.370635e-06,3.710442e-07,3.793092e-07,0.003982,6.166865e-07,2.140105e-05,7.279332e-06,4.331335e-01,8.852982e-08,5.078267e-03,1.332075e-05,1.244337e-03,9.567340e-07,4.653632e-07,1.475766e-06,9.152085e-02,7.723756e-06,5.001790e-03,4.007418e-05,1.769985e-06,1.694133e-03,7.004240e-08,2.161839e-04,1.159809e-05
1989,ffa5cf6d6,2.322441e-06,2.146701e-06,1.077083e-02,0.163286,7.166504e-07,4.062109e-06,3.300846e-07,2.779807e-04,1.162094e-08,1.258042e-03,3.702784e-07,1.912474e-06,1.011830e-03,2.940392e-07,7.960980e-08,2.735497e-01,1.448996e-06,2.866691e-02,2.261433e-08,3.385785e-07,1.951643e-06,3.396649e-06,2.182433e-06,1.355792e-05
1990,ffa88cbb8,8.721604e-09,5.612688e-08,2.491377e-06,0.831818,2.938613e-11,7.464962e-10,4.071380e-12,7.264311e-06,7.580243e-12,5.614509e-07,3.605737e-10,5.363798e-10,1.395037e-08,3.274442e-10,9.399098e-08,5.103131e-09,5.567571e-03,1.683662e-08,5.698992e-13,8.783185e-11,6.441617e-09,1.650725e-07,5.877637e-11,2.122599e-09


In [ ]:
device = torch.device("cuda")
model.eval().to(device)
preds = []

for path in tqdm(test_wav_pathes):
    y, sr = sf.read(path)
    mel_img = signal_to_mel(y, sr, config["dataset"]["params"])
#     mel_img = torch.from_numpy(mel_img).clone().to(device)
#     pred = model(mel_img)
# #         pred = torch.mean(pred['clipwise_output'], 0)
# #         pred = pred.to('cpu').detach().numpy().copy()
#     preds.append(pred)
    
    
    b_start = 0
    b_end = BATCH
    b_preds = []
    while(b_start < mel_img.shape[0]):
        if(b_end >= mel_img.shape[0]):
            mel_batch = mel_img[b_start:]
        else:
            mel_batch = mel_img[b_start:b_end]
        mel_batch = torch.from_numpy(mel_batch).clone().to(device)
        pred = model(mel_batch)
        pred = pred['framewise_output'].to('cpu').detach().numpy().copy()
        b_preds.extend(pred)
        b_start += BATCH
        b_end += BATCH
    b_preds = np.array(b_preds).reshape(-1, 24)
    pred = np.max(b_preds, axis=0)
    preds.append(pred)    
    
            
preds = np.array(preds)